In [2]:
import pandas as pd
import requests
# from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [10]:
!pip install torch
!pip install transformers

from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model = AutoModel.from_pretrained("monologg/kobert")

sentence = "오늘은 정말 기분이 좋은 날입니다."
inputs = tokenizer(sentence, return_tensors='pt')
outputs = model(**inputs)

print(outputs[0])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 15.8 MB/s eta 0:00:00


tensor([[[-0.2236,  0.0273,  0.1581,  ...,  0.0968, -0.0675,  0.0807],
         [ 0.1196, -0.1713,  0.2286,  ..., -0.1451, -0.4710,  0.0590],
         [ 0.2693, -0.0831,  0.4557,  ..., -0.2830, -0.3391, -0.1290],
         ...,
         [-0.1384, -0.1252,  0.4488,  ..., -0.0982, -0.0437, -0.1144],
         [-0.1732, -0.1893,  0.3521,  ..., -0.2993,  0.0260, -0.2094],
         [-0.0851, -0.0081,  0.0838,  ..., -0.2540, -0.5372,  0.1287]]],
       grad_fn=<NativeLayerNormBackward0>)


In [12]:
sentence = "오늘은 정말 기분이 좋은 날입니다."
inputs = tokenizer(sentence, return_tensors='pt')
outputs = model(**inputs)

outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.2236,  0.0273,  0.1581,  ...,  0.0968, -0.0675,  0.0807],
         [ 0.1196, -0.1713,  0.2286,  ..., -0.1451, -0.4710,  0.0590],
         [ 0.2693, -0.0831,  0.4557,  ..., -0.2830, -0.3391, -0.1290],
         ...,
         [-0.1384, -0.1252,  0.4488,  ..., -0.0982, -0.0437, -0.1144],
         [-0.1732, -0.1893,  0.3521,  ..., -0.2993,  0.0260, -0.2094],
         [-0.0851, -0.0081,  0.0838,  ..., -0.2540, -0.5372,  0.1287]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 4.9108e-04,  6.0109e-02, -2.0945e-01, -3.3830e-03, -9.1774e-01,
          9.9403e-01, -8.0837e-01,  7.8377e-03,  4.8179e-02, -1.6607e-02,
         -2.0984e-01, -3.9177e-02, -2.2142e-02, -1.1411e-01,  3.0750e-02,
          7.3569e-01, -9.9139e-01,  1.1706e-01, -2.7428e-02, -1.0112e-02,
          6.8423e-02, -1.8391e-02, -2.3961e-02,  4.3295e-01,  3.9690e-02,
          7.8904e-01, -9.0872e-01,  5.7587e-02, -8.8869e-01,  2.941

In [3]:
# 파일 URL 설정
url = 'https://raw.githubusercontent.com/kavgan/nlp-in-practice/master/data/Sentiment%20Analysis%20Dataset.csv'

# 파일 다운로드
response = requests.get(url)

# 파일 저장
with open('sentiment_data.csv', 'wb') as f:
    f.write(response.content)

In [4]:
# 데이터 불러오기
df = pd.read_csv('sentiment_data.csv')

In [5]:
df

,404: Not Found


In [ ]:
# 형태소 분석기 생성
okt = Okt()

# 형태소 분석 함수 정의
def tokenize(text):
    tokens = okt.pos(text, stem=True)
    return ['/'.join(t) for t in tokens if t[1] in ['Noun', 'Adjective']]

# 벡터화
cv = CountVectorizer(tokenizer=tokenize)
X = cv.fit_transform(df['text'])

# 분류기 학습
nb = MultinomialNB()
nb.fit(X, df['sentiment'])

# 예측 함수 정의
def predict_sentiment(text):
    X_new = cv.transform([text])
    y_pred = nb.predict(X_new)
    return y_pred[0]

# 감성 분석 테스트
texts = ['이 영화는 정말 재밌었어요!', '이 책은 너무 지루했어요.']
for text in texts:
    sentiment = predict_sentiment(text)
    print(f'{text} => {"긍정" if sentiment == 1 else "부정"}')